<h1>How have global energy production trends changed over time?</h1>

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

<h2>Goals</h2>
<p>By the end of this case, you should be very comfortable writing your own functions using <code>pandas</code> and applying them to entire datasets. You'll understand how functions work in Python, including anonymous functions (using the keyword <code>lambda</code>), and you'll feel comfortable analyzing and manipulating larger datasets. You'll also have gained experience with exploring a dataset that is only loosely organised and about which you have very little initial information.</p>

<h2>Introduction</h2>

<p><strong>Business Context.</strong> Global electricity production, consumption, import, and export is complex and interesting for a variety of reasons. Each country has to keep track of a vast array of information to ensure that they produce enough electricity, yet balance these needs against medium-term financial implications and environmental concerns.</p>
<p>You are an analyst working at a non-governmental organization (NGO) that reports on global energy trends. Your department has acquired a large CSV file, but your colleagues are battling to extract relevant insights from it using Excel due to its size and format. Worse still, it has thousands of variables and they are not sure which ones are interesting. Thus, you have been made responsible for supporting your team's journalists by providing them with data and insights that they can turn into written reports.</p>
<p><strong>Business Problem.</strong>  Your task is to <strong>break the available data down into smaller files, understand the information that is available, and extract key insights for an upcoming report on global power patterns.</strong> Specifically, your team wants you to answer the following questions:</p>
<ul>
<li>How much power is produced?</li>
<li>How much power is consumed?</li>
<li>How much power is imported and exported? </li>
<li>How much of this power is renewable?</li>
<li>How are these trends in production, consumption, import, and export changing over time?</li>
</ul>
<p><strong>Analytical Context.</strong> The data is stored in a large CSV file containing information on power production and consumption by country and year. You will: 1) break down the data into summarized CSV files to share with your colleagues; 2) manipulate the data to create more categories from the existing columns; 3) find the biggest players in different categories, including total energy export and total production by type (e.g. nuclear); and finally 4) find trends in the data, such as which countries have the fastest growing energy production.</p>

<h2>Getting started with the International Energy Statistics data</h2>
<p>The data file you have been given is a single CSV located at <code>data/all_energy_statistics.csv</code>. Your colleagues have informed you that the data is from http://data.un.org/Explorer.aspx, but they don't know much else about it. </p>
<p>They specifically note that the data is very ["narrow"] (https://en.wikipedia.org/wiki/Wide_and_narrow_data). Although the file contains data for a wide variety of things, such as "Total Energy Production" all the way through to "Additives and Oxygenates - Exports", it has very few columns. </p>
<p>Generally, when dealing with "wide" data, we can be fairly sure that all data in the same column is comparable. In this case, you'll notice a <code>unit</code> column. Not all numerical data in the <code>quantity</code> column is directly comparable. For example, sometimes the number in this column is defined in terms of "Metric tons, thousand" and sometimes in "Kilowatt-hours, million" -- evidently very different concepts!</p>
<p>As always, our first step is to read the data from disk and take a look at the first few rows:</p>

In [4]:
# load dataset

try:
  # Using google colab
  from google.colab import drive
  drive.mount('/content/drive')
  df = pd.read_csv("drive/MyDrive/data/all_energy_statistics.csv")
except:
  # if not...
  df = pd.read_csv("../datasets/all_energy_statistics.csv")

In [6]:
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates


<p>You'll notice that there is more of a delay than before when running the <code>read_csv</code> function. This dataset has over 1 million rows, so it takes a while to load it all into memory. From the first rows, we can immediately gain some useful insights</p>
<ul>
<li>The <code>category</code> column looks like it is well organized. All the samples we see are lowercase and underscores are used instead of spaces</li>
<li>The <code>commodity_transaction</code> column looks more like a human-readable description. We can see how it includes a description of the category (e.g. "additives_and_oxygenates" matches with "Additives and Oxygenates" and "wind_electricity" matches with "Electricity - ....wind....")</li>
<li>We see <code>year</code> ranges from at least 1995 to 2014 </li>
<li>As mentioned before, we'll need to be careful when comparing quantities, as the <code>unit</code> column might change the meaning of the <code>quantity</code> column.</li>
</ul>
<p>A good first question to ask is how many unique values there are for the following columns:</p>
<ul>
<li><code>country_or_area</code></li>
<li><code>commodity_transaction</code></li>
<li><code>year</code></li>
<li><code>category</code></li>
</ul>
<p>Let's find out:</p>

In [7]:
print(df.year.min())
print(df.year.max())
print("----------")
print("commodity_transaction")
print(df.commodity_transaction.unique())
print()
print("num unique values: ", len(df.commodity_transaction.unique()))
print()
print("----------")
print(df.category.unique())
print()
print("num unique values: ", len(df.category.unique()))
print()
print("---------------")
print(df.country_or_area.unique())
print()
print("num unique values: ", len(df.country_or_area.unique()))


1990
2014
----------
commodity_transaction
['Additives and Oxygenates - Exports' 'Additives and Oxygenates - Imports'
 'Additives and Oxygenates - Production' ...
 'White spirit and special boiling point industrial spirits - Transformation'
 'White spirit and special boiling point industrial spirits - Transformation in petrochemical plants'
 'Electricity - total wind production']

num unique values:  2452

----------
['additives_and_oxygenates' 'animal_waste' 'anthracite'
 'aviation_gasoline' 'bagasse' 'biodiesel' 'biogases' 'biogasoline'
 'bitumen' 'black_liquor' 'blast_furnace_gas' 'brown_coal_briquettes'
 'brown_coal' 'charcoal' 'coal_tar' 'coke_oven_coke' 'coking_coal'
 'conventional_crude_oil' 'direct_use_of_geothermal_heat'
 'direct_use_of_solar_thermal_heat'
 'electricity_net_installed_capacity_of_electric_power_plants' 'ethane'
 'falling_water' 'fuel_oil' 'fuelwood' 'gas_coke' 'gas_oil_diesel_oil'
 'gasoline_type_jet_fuel' 'gasworks_gas' 'geothermal' 'hard_coal' 'heat'
 'hydro'

<p>We can see that <code>country_or_area</code> has 243 unique values, more than the officially recognised 195, because this list includes some former countries such as the USSR as well as areas like Antarctic Fisheries which are not formal countries.</p>
<p>As expected, the <code>categories</code> column is well standardized and breaks each row into one of 71 unique categories, while the <code>commodity_transaction</code> row is slightly more chaotic and consists of 2452 unique values.</p>
<p>In terms of time, our data ranges from 1990 - 2014 inclusive, so 25 years in total.</p>
<p>Note that the output of <code>unique()</code> is automatically truncated for large lists, with a <code>...</code> inserted to indicate this.</p>
<p>Since the <code>commodity_transaction</code> column is a bit chaotic, we'll need to touch it up a bit. Let's create a copy of our dataframe before we start changing it so we can refer back to the original values if necessary.</p>

In [8]:
df_orig = df.copy()

<p>The first thing we noticed about the <code>commodity_transaction</code> column is that it uses hyphens (<code>-</code>) as separators. We can also see that it uses lowercase and capital letters - often something that makes analysis harder if we are going to do any string matching (e.g. find the word "production", which might skip descriptions which use "Production" instead). </p>
<p>Let's start by lowercasing all of the descriptions. In the previous case, you learned how to do this by creating a separate list, looping through the dataframe, and then adding all the items from the list as a new column. We could achieve what we wanted as follows:</p>

In [9]:
%%time
clean_transaction_list = []

for item in df['commodity_transaction']:
    item = item.lower()
    clean_transaction_list.append(item)

df['clean_transaction'] = clean_transaction_list

CPU times: total: 1.06 s
Wall time: 1.51 s


In [10]:
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,additives and oxygenates - exports


<p>We added <code>%%time</code> at the top of our cell to make Jupyter output information about how long it took to run that cell. We can see that looping through our DataFrame and adding the column took nearly 1 second to complete. It also took 5 lines of code.</p>
<p>As its very common to need to apply the same operation on every row of a dataset, <code>pandas</code> provides a shortcut to do this. You can use the <code>.apply()</code> function on a DataFrame directly and pass in a function to apply to every row. This is more efficient in two ways:</p>
<ul>
<li>It takes fewer lines of code, so it's faster to write the code (and to read it)</li>
<li><code>apply()</code> is optimized to take advantage of modern CPU features such as vectorization, so it runs in less time</li>
</ul>
<p>We can achieve exactly the same result as we did with our <code>for</code> loop using the <code>apply()</code> function as follows:</p>

In [11]:
%%time
df['clean_transaction2'] = df['commodity_transaction'].apply(str.lower)

CPU times: total: 250 ms
Wall time: 505 ms


In [12]:
%%time
df['clean_transaction2'] = df.commodity_transaction.apply(str.lower)

CPU times: total: 266 ms
Wall time: 525 ms


In [13]:
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,clean_transaction,clean_transaction2
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,additives and oxygenates - exports,additives and oxygenates - exports


<p>Here we can see that <code>.apply()</code> ran around twice as quickly as the iterative version and produced the same results (the <code>clean_transaction</code> and <code>clean_transaction2</code> columns are the same). You can read more about the <code>apply()</code> function <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html">here</a>, but in essence you call it from a column of a DataFrame and pass in a function. It applies that function to every row of that column in the DataFrame. In this case, we passed in the <code>str.lower</code> function, which converts a string to lowercase.</p>

<h2>Pre-processing and pivoting our data</h2>

<p>We noted before that the <code>commodity_transaction</code> column seemed to use hyphens to separate different concepts in a single column. Let's do some more analysis to see if this is true across the board.</p>

<h3>Exercise 1:</h3>
<p>Find out how many of the 2,000+ unique columns contain:</p>
<ul>
<li>0 hyphens</li>
<li>exactly 1 hyphen</li>
<li>more than 1 hyphen</li>
</ul>
<p><strong>Hint:</strong> You can use Python's built-in <a href="https://www.w3schools.com/python/ref_string_count.asp"><code>count()</code></a> method to count the occurrences of a character in a string).</p>

**Answer.**

In [14]:
hyphens_0 = 0
hyphens_1 = 0
hyphens_2plus = 0

for value in df.commodity_transaction.unique():
    hyphen_count = value.count("-")
    if hyphen_count == 0:
        hyphens_0 += 1
    elif value.count("-") == 1:
        hyphens_1 += 1
    else:
        hyphens_2plus += 1

print("zero hyphens", hyphens_0)
print("one hyphen", hyphens_1)
print("two or more hyphens", hyphens_2plus)

zero hyphens 57
one hyphen 1845
two or more hyphens 550


-------

<h3>Exercise 2:</h3>
<p>Write code to print out all descriptions with zero hyphens. What do you notice about these?</p>

**Answer.**

In [17]:
# Add a helper column 
df["hyphen_count"] = df["commodity_transaction"].str.count("-")

# Filter rows where the count is 0
no_hyphen_rows = df[df["hyphen_count"] == 0]

# Look at the distinct descriptions to understand what they are
unique_no_hyphen = (
    no_hyphen_rows["commodity_transaction"]
    .drop_duplicates()
    .sort_values()
    .to_list()
)

print(f"{len(unique_no_hyphen)} unique descriptions with no hyphen:")
for desc in unique_no_hyphen:
    print(" •", desc)


57 unique descriptions with no hyphen:
 • From chemical heat – Autoproducer
 • From chemical heat – Autoproducer – CHP plants
 • From chemical heat – Autoproducer – Electricity plants
 • From chemical heat – Main activity
 • From chemical heat – Main activity – CHP plants
 • From chemical heat – Main activity – Electricity plants
 • From chemical sources – Autoproducer
 • From chemical sources – Autoproducer – CHP plants
 • From chemical sources – Autoproducer – Heat plants
 • From combustible fuels – Autoproducer
 • From combustible fuels – Autoproducer – CHP plants
 • From combustible fuels – Autoproducer – Electricity plants
 • From combustible fuels – Autoproducer – Heat plants
 • From combustible fuels – Main activity
 • From combustible fuels – Main activity – CHP plants
 • From combustible fuels – Main activity – Electricity plants
 • From combustible fuels – Main activity – Heat plants
 • From electric boilers – Main activity
 • From heat pumps – Main activity
 • From other sou

-------

<h3>Passing our own functions to <code>apply()</code></h3>
<p>We previously passed the built-in <code>str.lower()</code> function to the <code>apply()</code> function to apply to it every row in our DataFrame. Now we want to clean up the m-dashes and lowercase the result at the same time. Let's write our own custom Python function to do both, and pass that to <code>apply()</code> instead. You can read more about writing your own custom functions in Python <a href="https://www.w3schools.com/python/python_functions.asp">here</a>:</p>

In [18]:
def clean_transaction_description(transaction_description):
    """Lowercase the input and replace all m-dashes with hyphens"""
    clean = transaction_description.lower()
    clean = clean.replace("–", "-")
    return clean


# drop the columns we added before so we can recreate them with our new clean function
df = df.drop(columns=['clean_transaction', 'clean_transaction2'])
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,hyphen_count
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,1
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,1
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,1
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,1
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,1


In [19]:
df['clean_transaction'] = df['commodity_transaction'].apply(clean_transaction_description)
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,hyphen_count,clean_transaction
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,1,additives and oxygenates - exports
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,1,additives and oxygenates - exports
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,1,additives and oxygenates - exports
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,1,additives and oxygenates - exports
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,1,additives and oxygenates - exports


<p>Here we used <code>apply()</code> again, but this time passed in our own function which did both the lowercasing and the replacing of m-dashes with hyphens.</p>
<p>We've now seen how to use the <code>apply()</code> function with both built-in functions and our own custom functions. There's one more way we can use <code>apply()</code> though: with custom <strong>anonymous functions</strong> using the Python <code>lambda</code> keyword. Let's see how to achieve the same result using <code>lambda</code>:</p>

In [20]:
df = df.drop(columns=['clean_transaction'])
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,hyphen_count
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,1
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,1
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,1
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,1
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,1


In [21]:
# lowercase the description and replace m-dashes with hyphens in one line
df['clean_transaction'] = df['commodity_transaction'].apply(lambda x: x.lower().replace("–", "-"))
df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,hyphen_count,clean_transaction
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,1,additives and oxygenates - exports
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,1,additives and oxygenates - exports
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,1,additives and oxygenates - exports
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,1,additives and oxygenates - exports
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,1,additives and oxygenates - exports


<p>This code is functionally equivalent to what we ran before, but it's more concise. Instead of giving our function a name (<code>clean_transaction_description</code>), we can declare an anonymous function by using the <a href="https://www.w3schools.com/python/python_lambda.asp"><code>lambda</code></a> keyword. This says that we are going to pass in a series of <code>x</code> values (the descriptions), and describes what to do to each of them. The advantage of doing this is that it's more concise. The disadvantage is that it can be harder to read and it prevents us from using our function again later without redefining it all over again.</p>

<h3>Extracting the most interesting rows</h3>
<p>It's hard to manually inspect over 2,000 unique description values, but we know that we're mainly interested in: </p>
<ul>
<li>Import</li>
<li>Export</li>
<li>Total production</li>
<li>Total demand or consumption</li>
<li>Renewables</li>
</ul>
<p>We can search for some keywords in the descriptions using code similar to the following:</p>

In [22]:
[x for x in df['clean_transaction'].unique() if "import" in x]

['additives and oxygenates - imports',
 'anthracite - imports',
 'aviation gasoline - imports',
 'biodiesel - imports',
 'biogases - imports',
 'biogasoline - imports',
 'bitumen - imports',
 'brown coal briquettes - imports',
 'brown coal - imports',
 'charcoal - imports',
 'coal tar - imports',
 'coking coal - imports',
 'conventional crude oil - imports',
 'ethane - imports',
 'fuel oil - imports',
 'fuelwood - imports',
 'gas coke - imports',
 'gas oil/ diesel oil - imports',
 'gasoline-type jet fuel - imports',
 'gasworks gas - imports',
 'hard coal - imports',
 'heat - imports',
 'industrial waste - imports',
 'kerosene-type jet fuel - imports',
 'lignite - imports',
 'liquefied petroleum gas (lpg) - imports',
 'lubricants - imports',
 'motor gasoline - imports',
 'municipal wastes - imports',
 'naphtha - imports',
 'natural gas (including lng) - imports',
 'natural gas liquids - imports',
 'of which: biodiesel - imports',
 'of which: biogasoline - imports',
 'oil shale - imports

<p>This gives us a much more manageable list to look through, and we can see that "electricity - imports" is likely an interesting value. We can cross-check this in the main dataset (and see all columns to boot) as follows:</p>

In [23]:
## Note the below is functionally equivalent to
# df[df["clean_transaction"] == "electricity - imports"].head()
# but slightly easier to type

df[df.clean_transaction == "electricity - imports"].head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,hyphen_count,clean_transaction
1108326,Afghanistan,Electricity - imports,2014,"Kilowatt-hours, million",3710.8,NaN,total_electricity,1,electricity - imports
1108327,Afghanistan,Electricity - imports,2013,"Kilowatt-hours, million",3615.2,NaN,total_electricity,1,electricity - imports
1108328,Afghanistan,Electricity - imports,2012,"Kilowatt-hours, million",3071.0,NaN,total_electricity,1,electricity - imports
1108329,Afghanistan,Electricity - imports,2011,"Kilowatt-hours, million",2732.0,NaN,total_electricity,1,electricity - imports
1108330,Afghanistan,Electricity - imports,2010,"Kilowatt-hours, million",1867.0,NaN,total_electricity,1,electricity - imports


<h3>Exercise 3:</h3>
<p>Use the above method or any other method that you prefer to explore the transaction descriptions and define a Python list containing the 9 most interesting ones. These should cover the total values for import, export, total production, total demand, and renewable energy production.</p>

**Answer.**

In [75]:
# -----------------------------------------
# Columna que contiene las descripciones
# -----------------------------------------
col = "clean_transaction"

# Todas las descripciones únicas, ya en minúsculas para facilitar el filtrado
unique_txns = (
    df[col]
    .dropna()
    .str.strip()
    .str.lower()
    .drop_duplicates()
    .sort_values()
)

# -----------------------------------------
# Patrones clave → 9 columnas de interés
# -----------------------------------------
kw_map = {
    "demand"      : r"total final consumption",      # demanda / consumo final
    "production"  : r"^total.*production$",          # producción total
    "imports"     : r"imports$",                     # importaciones
    "exports"     : r"exports$",                     # exportaciones
    "hydro"       : r"hydro",                        # hidroeléctrica
    "wind"        : r"wind",                         # eólica
    "solar"       : r"solar",                        # solar
    "geothermal"  : r"geothermal",                   # geotérmica
    "tide"        : r"(tide|wave|ocean)",            # mareomotriz / olas
}

# Tomamos el *primer* match de cada patrón. 
# (Si tu dataset tiene varias variantes para una misma fuente,
#   ajusta el regex o usa .iloc para coger la que prefieras.)
keep_values = []
for label, pattern in kw_map.items():
    hit = unique_txns[unique_txns.str.contains(pattern, regex=True, case=False)]
    if not hit.empty:
        keep_values.append(hit.iloc[0])
    else:
        print(f"No se encontró ninguna descripción para: {label}")

print("Descripciones seleccionadas:")
for d in keep_values:
    print(" •", d)


⚠️  No se encontró ninguna descripción para: demand
⚠️  No se encontró ninguna descripción para: production
Descripciones seleccionadas:
 • additives and oxygenates - imports
 • additives and oxygenates - exports
 • electricity - net installed capacity of electric power plants, public hydro
 • electricity - net installed capacity of electric power plants, public wind
 • direct use of solar thermal heat - consumption by agriculture, forestry and fishing
 • direct use of geothermal heat - consumption by agriculture, forestry and fishing
 • electricity - net installed capacity of electric power plants, public tide, wave, marine


C:\Users\ljpca\AppData\Local\Temp\ipykernel_18084\4164636604.py:36: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  hit = unique_txns[unique_txns.str.contains(pattern, regex=True, case=False)]


In [74]:
df['clean_transaction'].unique()

array(['additives and oxygenates - exports',
       'additives and oxygenates - imports',
       'additives and oxygenates - production', ...,
       'white spirit and special boiling point industrial spirits - transformation',
       'white spirit and special boiling point industrial spirits - transformation in petrochemical plants',
       'electricity - total wind production'], dtype=object)

-------

<h3>Pivoting the interesting values into their own columns</h3>
<p>Of course, now that we've identified the most interesting transaction descriptions, we probably ought to pull them out of that single column that they're stuck in. Let's "pivot" our data to a more useable format, keeping each of these interesting values as new columns. This translates our data from a fairly narrow format into a wider one.</p>
<p>You might know of "pivot tables" from Excel. If not, don't worry - you'll come across them later and in more detail. But if you do know of them, you'll recognize that this pretty much the exact same thing. We'll use the pivot function in pandas, which you can read more about <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html">here</a>. For now, just try to understand how the following code works, but you won't be expected to do this yourself until you've gained more experience with <code>pandas</code>:</p>

In [76]:
# we'll keep our "interesting" values after we turn them into columsn
# but we'll also keep the "country" and "year" columns
final_keep_values = ["country_or_area", "year"] + keep_values

# Turn values in the 'commodity transaction' column
# into our new column names
# and keep only the 'quantity' column as the new values
df_countries = pd.pivot_table(
    df,
    values="quantity",
    index=["country_or_area", "year"],
    columns="commodity_transaction",
).reset_index()[final_keep_values]

# rename the columns to be more concise
df_countries.columns = [
    "country",
    "year",
    "demand",
    "production",
    "imports",
    "exports",
    "hydro",
    "wind",
    "solar",
    "geothermal",
    "tide",
]

# output with the energy production leaders first
df_countries.sort_values(by="production", ascending=False)

KeyError: "['additives and oxygenates - imports', 'additives and oxygenates - exports', 'electricity - net installed capacity of electric power plants, public hydro', 'electricity - net installed capacity of electric power plants, public wind', 'direct use of solar thermal heat - consumption by agriculture, forestry and fishing', 'direct use of geothermal heat - consumption by agriculture, forestry and fishing', 'electricity - net installed capacity of electric power plants, public tide, wave, marine'] not in index"

In [ ]:
df_countries.head()

In [66]:
df_countries['year'].value_counts()

year
2014    229
2013    229
2012    229
2007    227
2011    226
2010    226
2009    226
2008    226
2006    226
2005    226
2003    225
2004    225
2002    225
2001    223
1997    223
1996    223
1995    223
2000    222
1999    222
1998    222
1994    222
1993    222
1992    222
1990    200
1991    199
Name: count, dtype: int64

In [67]:
df_countries['hydro'].unique()

array([ 764. ,  690. ,  478. , ..., 5201.8, 5387.3, 4981.8])

<p>As expected, in earlier years, we have data for fewer countries.</p>
<p>The final check we should do is whether any of the values we kept used a different "unit". A quick scan of the data shows that all of the values we are interested in are measured in "Kilowatt-hours, million", but it's possible that some small values could be measured as "Kilowatt-hours, thousand", for example. Let's look for unique values used in our <code>keep_values</code> list:</p>

In [68]:
x = keep_values[0]
all_units = []

for value in keep_values:
    units_used = list(df[df.commodity_transaction == value]['unit'].unique())
    all_units += units_used
print(set(all_units))

{'Kilowatt-hours, million', 'Terajoules'}


<p>All good! Only one unit is used. So we are done with data preparation and we can start exploring our dataset for information.</p>

<h2>Exploring growth of power production and renewables</h2>

<p>As mentioned, the team is interested in analyzing countries based on their renewable energy production. We currently know how much power they produce in total and how much of this is due to each of a number of renewable options. We'll start by adding some supplementary data and then analyzing our dataset for interesting countries and patterns.</p>

<h3>Exercise 4:</h3>
<p>Add a new summary column called <code>renewable_percent</code> which gives the percentage of total power production which is made up of renewable energy.</p>
<p><strong>Hint:</strong> You might notice that some values are <code>na</code>, meaning <code>not available</code>. We can probably assume that these are 0 (though this might not always be meaningful; e.g. if we don't have data on the USSR in 2014, it's not because its power plants are all turned off!). You can use the <code>pandas</code> <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html"><code>fillna</code></a> method to replace <code>na</code> values with 0.</p>

**Answer.**

In [69]:
# Identify the columns that represent renewable generation
renew_cols = ["hydro", "wind", "solar", "geothermal", "tide"]

# Treat missing generation as zero (assumption per the hint)
df_countries[renew_cols] = df_countries[renew_cols].fillna(0)

# Compute total renewables and the share vs. total production
df_countries["renewables_total"] = df_countries[renew_cols].sum(axis=1)

# Avoid divide-by-zero; if production is 0 or NaN, set share to 0
df_countries["renewable_percent"] = np.where(
    df_countries["production"].fillna(0) == 0,
    0,
    (df_countries["renewables_total"] / df_countries["production"]) * 100,
).round(2)   # keep two decimals for readability

# Quick peek
df_countries[["country", "year", "production", "renewables_total", "renewable_percent"]].head()
df_countries[df_countries['renewable_percent'] > 0 ][["country", "year", "production", "renewables_total", "renewable_percent"]].head()


commodity_transaction,country,year,production,renewables_total,renewable_percent


In [70]:
df_countries[["country", "year", "production", "renewables_total", "renewable_percent"]].head()

commodity_transaction,country,year,production,renewables_total,renewable_percent
0,Afghanistan,1990,NaN,764.0,0.0
1,Afghanistan,1991,NaN,690.0,0.0
2,Afghanistan,1992,NaN,478.0,0.0
3,Afghanistan,1993,NaN,475.0,0.0
4,Afghanistan,1994,NaN,472.0,0.0


-------

<h3>Exercise 5:</h3>
<p>Considering only the most recent year that we have data for (2014), which 5 countries produced the largest proportion of their power through renewables, and which 5 countries produced the smallest proportion of their power through renewables?</p>
<p><strong>Hint:</strong> You can use the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html"><code>sort_values</code></a> method in <code>pandas</code> to sort a DataFrame by a specific column, either descending or ascending.</p>

**Answer.**

-------

<h3>Question:</h3>
<p>Why do you think we are seeing a lot of very small countries on both lists?</p>

<p>Very small countries are not particularly representative of the global renewable power situation, so your team asks you to restrict your analysis only to countries that produce a lot of power.</p>

<h3>Exercise 6:</h3>
<p>Repeat the above analysis but only look at the countries in the top 10% of total power production.</p>
<p><strong>Hint:</strong> You can filter a DataFrame with multiple conditions by using the <code>&amp;</code> symbol; e.g.:</p>
<p><code>df_countries[df_countries.year == 2014 &amp; df_countries.wind &gt; 0]</code> </p>
<p>would give you a DataFrame of all countries in 2014 which had produced at least some wind power.</p>

**Answer.**

-------

<p>Of course, your team is also interested in looking at change in renewable energy over time. Let's look at the top and bottom 5 countries where the percentage of renewable energy they produced in 2014 is <strong>very different</strong> from the percentage in 1990.</p>

<h3>Exercise 7:</h3>
<p>Add a new column to your DataFrame which displays the difference in percentage renewable energy production between 2014 and 1990. Which are the top and bottom 5 countries? What do you notice about these countries? Perform this analysis both with all countries and again with only those in the 10% of total power production.</p>
<p><strong>Hint:</strong> you can use the <code>pivot()</code> method again to create a DataFrame which has 1990 and 2014 as columns and <code>renewable_percent</code> as values to help with this by using the following code</p>
<p><code>renewable_change = pd.pivot_table(
    df_countries, values="renewable_percent", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]</code></p>

**Answer.**

-------

<h3>Exercise 8:</h3>
<p>Your team is also interested in countries which are producing a lot more power now than they were 25 years ago. What are the top and bottom 10 countries in terms of growth of:</p>
<ul>
<li>Total power</li>
<li>Renewable power</li>
</ul>
<p>Note that because many countries were producing zero or very little renewable energy in 1990, doing a basic growth calculation will show that many countries have "infinite" (represented as <code>inf</code> in <code>pandas</code>) growth. To avoid this, restrict your results to countries which produced at least 1,000 units of renewable power in 1990 for the renewable growth analysis and at least 1,000 units of total power for the total growth analysis.</p>
<p><strong>Hint:</strong> Assuming you add a column called <code>renewable_total</code>, you can use the following pivots to generate tables similar to before for both renewable growth and total growth:</p>
<p><code>renewable_growth = pd.pivot_table(
    df_countries, values="renewable_total", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]</code></p>
<p><code>total_growth = pd.pivot_table(
    df_countries, values="production", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]</code></p>

**Answer.**

-------

<h3>Exercise 9:</h3>
<p>Finally, your team wants an easy-to-read label for each country based on total growth. They have given you the following specification for how the countries should be labeled:</p>
<ul>
<li>zero or negative growth = "No growth"</li>
<li>1% -100% growth = "Growing"</li>
<li>over 100% growth = "Growing fast"</li>
<li>NaN (if the data from 1990 or 2014 is NaN) = "Not Applicable"</li>
</ul>
<p>Calculate the label for each country, using the <code>apply()</code> method for efficiency. </p>
<p><strong>Hint:</strong> You can check if the value of variable <code>x</code> is Nan as follows:</p>
<p><code>import numpy as np
np.isnan(x)</code></p>

**Answer.**

-------

<h2>Largest importers and exporters of energy</h2>
<p>The final thing that your team wants to look into is imports and exports of energy by country.</p>

<h3>Exercise 10:</h3>
<p>Your team wants to know:</p>
<ul>
<li>Which countries have imported and exported the most power in total</li>
<li>Which countries have imported the largest percentage of their <em>demand</em> and exported the largest percentage of their <em>production</em></li>
</ul>
<p>Do the analysis for all countries <em>and</em> for only countries with total production in the top 10%.</p>

**Answer.**

-------

<h2>Writing new country-specific summary data to disk</h2>

<p>Your team is delighted that you've managed to make sense of the data and extract some insights. They want to explore the data themselves too, but all of their existing tools are designed to analyze data from only one country at a time. They have asked that you create separate CSV files for each country, using the country as the file name, with a maximum of 25 rows per file (one per year) and columns for imports, exports, etc.</p>
<p>To do this, we use the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html"><code>to_csv</code></a> function on a given DataFrame to write it to a file. We create a new directory called "output_csvs" in our working directory so that we don't clutter up our workspace with 243 CSV files. Then we run the following code to write our data to disk:</p>

In [ ]:
OUTPUT_DIRECTORY = "output_csvs"

if not os.path.exists(OUTPUT_DIRECTORY):
    os.makedirs(OUTPUT_DIRECTORY)

for country in df_countries['country'].unique():
    country_df = df_countries[df_countries.country == country].drop(columns='country')
    country_df.to_csv(f"{OUTPUT_DIRECTORY}/{country}.csv")

<h2>Conclusions</h2>
<p>We saw a number of interesting trends in the global energy industry. Specifically, we saw that many countries are relying more and more on renewables, but that some of the countries with fast-growing demand are forced to turn to non-renewable sources to keep up.</p>
<p>We also noticed that contrary to our expectations of some countries being "net importers" and others being "net exporters" of power, many countries actually both import <em>and</em> export large amounts of power.</p>

<h2>Takeaways</h2>
<p>In this case, we covered some more features of <code>pandas</code> and got more practice with the features we covered previously. Specifically we saw how to:</p>
<ul>
<li>Use the <code>apply()</code> method in <code>pandas</code> with build-in functions, custom functions, and anonymous functions</li>
<li>Work with large datasets and explore these using basic string matching to find interesting columns, and reformat the results into more convenient formats</li>
<li>Pivot between wide and narrow formats</li>
<li>Plot basic line plots</li>
<li>Break up a large dataset into smaller ones and write these back to disk</li>
</ul>
<p>While you'll learn more advanced functionality than this in later cases, these basics will be used again and again, so keep coming back to this case as reference material as often as you need.</p>